In [5]:
import sys
sys.path.append("..")

from scripts.utils import DuckdbUtils

duckdb_utils = DuckdbUtils()

In [111]:
sql_query = """
SELECT DISTINCT
    violation_county
FROM
    silver_valid_violation_tickets
"""

duckdb_utils.run_sql_query_and_return_df(sql_query)

,violation_county
0,ST
1,QN
2,NY
3,BK
4,BX
5,Qns
6,None
7,Rich
8,K
9,Bronx


In [112]:
sql_query = """
SELECT DISTINCT
    violation_county
FROM
    silver_valid_violation_tickets
"""

duckdb_utils.run_sql_query_and_return_df(sql_query)

,violation_county
0,K
1,Queens
2,Kings
3,Bronx
4,ST
5,QNS
6,KINGS
7,R
8,NY
9,None


In [6]:
sql_query = """
-- 1. Revenue and Precinct Data
-- Check total revenue and count by precinct, focusing on Precinct 0
SELECT 
    violation_precinct,
    COUNT(*) AS ticket_count,
    SUM(fee_usd) AS total_revenue
FROM silver_valid_violation_tickets
GROUP BY violation_precinct
ORDER BY total_revenue DESC;
"""

duckdb_utils.run_sql_query_and_return_df(sql_query)

,violation_precinct,ticket_count,total_revenue
0,19,1466,112505.0
1,14,974,84530.0
2,13,1041,80945.0
3,6,1031,76835.0
4,114,1131,76055.0
...,...,...,...
72,121,122,8945.0
73,101,80,6160.0
74,100,70,5140.0
75,123,45,3565.0


Values of Interest:
We can now use the lineage to give us an understanding of which tables and values to profile first
- All Gold metrics of interest are `silver_valid_violation_tickets` and `silver_parking_violation_codes`, thus those two tables will be our starting point.
- `silver_valid_violation_tickets`
    - `violation_precinct`
    - `fee_usd`
    - `violation_code`
    - `violation_county`
    - `issuing_agency`
- `silver_parking_violation_codes`
    - `violation_code`
    - `definition`

We also want to explore these value ***one node** before a table in the data lineage to see if there is a change in the values. Then repeat this process until we find a difference or reach the earliest layer of ingestion. If we see a difference between two tables, it typically means its a business logic error in the SQL. If it's seen all the way to the ingestion layer, it implies that the data itself has errors and you must work with upstream stakeholders to resolve and or implement business logic to resolve the issue downstream as a band-aid solution (i.e. take on tech debt).
- `silver_valid_violation_tickets` -> `silver_violation_tickets`
- `silver_violation_tickets` -> `silver_parking_violation_codes`
- `silver_violation_tickets` -> `silver_parking_violations`
- `silver_parking_violation_codes` -> `bronze_parking_violation_codes` (ingestion layer)
- `silver_parking_violations` -> `bronze_parking_violations` (ingestion layer)

In [82]:
sql_query = """
WITH silver_violation_tickets_dq_check AS (
    SELECT DISTINCT
        issuing_agency,
        '3.silver_violation_tickets' AS table_name
    FROM
        silver_violation_tickets
    WHERE
        EXTRACT(year FROM issue_date) == 2023 AND
        EXTRACT(month FROM issue_date) <= 8
),

silver_valid_violation_tickets_dq_check AS (
    SELECT DISTINCT
        issuing_agency,
        '4.silver_valid_violation_tickets' AS table_name
    FROM
        silver_valid_violation_tickets
    WHERE
        EXTRACT(year FROM issue_date) == 2023 AND
        EXTRACT(month FROM issue_date) <= 8
)

SELECT
    silver_violation_tickets_dq_check.issuing_agency,
    silver_violation_tickets_dq_check.table_name AS silver_violation_tickets,
    silver_valid_violation_tickets_dq_check.table_name AS silver_valid_violation_tickets
FROM
    silver_violation_tickets_dq_check
LEFT JOIN
    silver_valid_violation_tickets_dq_check ON
        silver_violation_tickets_dq_check.issuing_agency = silver_valid_violation_tickets_dq_check.issuing_agency
"""

duckdb_utils.run_sql_query_and_return_df(sql_query)

,issuing_agency,silver_violation_tickets,silver_valid_violation_tickets
0,X,3.silver_violation_tickets,4.silver_valid_violation_tickets
1,R,3.silver_violation_tickets,4.silver_valid_violation_tickets
2,Y,3.silver_violation_tickets,4.silver_valid_violation_tickets
3,A,3.silver_violation_tickets,4.silver_valid_violation_tickets
4,S,3.silver_violation_tickets,4.silver_valid_violation_tickets
5,W,3.silver_violation_tickets,4.silver_valid_violation_tickets
6,4,3.silver_violation_tickets,4.silver_valid_violation_tickets
7,C,3.silver_violation_tickets,4.silver_valid_violation_tickets
8,F,3.silver_violation_tickets,4.silver_valid_violation_tickets
9,M,3.silver_violation_tickets,4.silver_valid_violation_tickets


In [98]:
sql_query = """
SELECT
    issuing_agency,
    'All tickets' AS filter_type,
    COUNT(issuing_agency) AS ticket_count
FROM
    silver_violation_tickets
WHERE
    EXTRACT(year FROM issue_date) == 2023 AND
    EXTRACT(month FROM issue_date) <= 8 AND
    issuing_agency = 'V'
GROUP BY
    issuing_agency

UNION

SELECT
    issuing_agency,
    'Only Precinct 0 Tickets' AS filter_type,
    COUNT(issuing_agency) AS ticket_count

FROM
    silver_violation_tickets
WHERE
    EXTRACT(year FROM issue_date) == 2023 AND
    EXTRACT(month FROM issue_date) <= 8 AND
    issuing_agency = 'V' AND
    issuer_precinct = 0
GROUP BY
    issuing_agency
"""

duckdb_utils.run_sql_query_and_return_df(sql_query)

,issuing_agency,filter_type,ticket_count
0,V,Only Precinct 0 Tickets,23890
1,V,All tickets,23890


In [102]:
sql_query = """
SELECT DISTINCT
    issuing_agency,
    COUNT(issuing_agency) AS ticket_count
FROM
    silver_violation_tickets
WHERE
    EXTRACT(year FROM issue_date) == 2023 AND
    EXTRACT(month FROM issue_date) <= 8 -- AND
    -- issuer_precinct = 0
GROUP BY
    issuing_agency
ORDER BY
    ticket_count DESC
"""

duckdb_utils.run_sql_query_and_return_df(sql_query)

,issuing_agency,ticket_count
0,T,25066
1,V,21469
2,XYZ,5488
3,S,1728
4,P,840
5,K,46
6,M,18
7,Y,12
8,A,12
9,8,10


In [79]:
sql_query = """
SELECT
    silver_violation_tickets.violation_code,
    silver_parking_violation_codes.definition,
    COUNT(silver_violation_tickets.violation_code) AS ticket_count
FROM
    silver_violation_tickets
LEFT JOIN
    silver_parking_violation_codes ON
        silver_violation_tickets.violation_code = silver_parking_violation_codes.violation_code
WHERE
    violation_precinct = 0
GROUP BY
    silver_violation_tickets.violation_code,
    silver_parking_violation_codes.definition
ORDER BY
    ticket_count DESC;
"""

duckdb_utils.run_sql_query_and_return_df(sql_query)

,violation_code,definition,ticket_count
0,36,PHTO SCHOOL ZN SPEED VIOLATION,74128
1,7,FAILURE TO STOP AT RED LIGHT,8110
2,5,BUS LANE VIOLATION,6954
3,12,MOBILE BUS LANE VIOLATION,1308
4,21,NO PARKING-STREET CLEANING,24
5,40,FIRE HYDRANT,20
6,20,NO PARKING-DAY/TIME LIMITS,6
7,67,PEDESTRIAN RAMP,6
8,71,INSP. STICKER-EXPIRED/MISSING,6
9,14,NO STANDING-DAY/TIME LIMITS,6


In [2]:
import pandas as pd
import numpy as np

# Load your original parking violations dataset
violations_df = pd.read_csv("../data/part_1_clean_data/parking_violations_issued_fiscal_year_2023_sample.csv")

# Determine the proportion of rows to alter (e.g., 1% of data)
error_fraction = 0.10

# Randomly select rows to introduce invalid issuing agency codes
num_rows_to_alter = int(len(violations_df) * error_fraction)
rows_to_alter = violations_df.sample(n=num_rows_to_alter, random_state=42).index

# Inject invalid agency code 'XYZ'
violations_df.loc[rows_to_alter, 'Issuing Agency'] = 'XYZ'

print(f"Injected invalid agency codes into {num_rows_to_alter} rows.\n")

# Quick verification: count tickets per issuing agency
agency_counts = violations_df.groupby('Issuing Agency').size().reset_index(name='ticket_count')

# Display the agency counts
print("Ticket counts by Issuing Agency after injection:\n")
print(agency_counts)

Injected invalid agency codes into 10000 rows.

Ticket counts by Issuing Agency after injection:

   Issuing Agency  ticket_count
0               3            27
1               4             1
2               8            13
3               9             4
4               A            22
5               B             1
6               C             4
7               F             3
8               G             1
9               K            93
10              L             7
11              M            23
12              N             3
13              O             8
14              P          1623
15              Q             2
16              R             8
17              S          3128
18              T         44265
19              U             9
20              V         40711
21              W             4
22              X            19
23            XYZ         10000
24              Y            21


In [3]:
# Define realistic, moderate county inconsistencies
moderate_county_mapping = {
    'Kings': ["King's", 'KINGS'],
    'Qns': ['Queens', 'QNS'],
    'Bx': ['Bronx', 'BX']
}

# Function to introduce moderate complexity inconsistency
def inject_county_inconsistency(county):
    if county in moderate_county_mapping:
        # 50% chance to replace with a random variant
        if np.random.rand() < 0.5:
            return np.random.choice(moderate_county_mapping[county])
    return county

# Apply moderate complexity inconsistency to 'Violation County'
violations_df['Violation County'] = violations_df['Violation County'].apply(inject_county_inconsistency)

print("Moderately complex county inconsistencies successfully injected.\n")

# Quick verification
county_counts = violations_df.groupby('Violation County').size().reset_index(name='ticket_count')
print("Ticket counts by Violation County after moderate injection:\n")
print(county_counts)

Moderately complex county inconsistencies successfully injected.

Ticket counts by Violation County after moderate injection:

   Violation County  ticket_count
0                BK         14399
1                BX         12815
2             Bronx           885
3                 K         12286
4             KINGS           412
5            King's           387
6             Kings           850
7                MN          4874
8                NY         21121
9                 Q         11089
10               QN         16044
11              QNS           115
12              Qns           240
13           Queens           119
14                R           905
15             Rich            10
16               ST          2880


In [4]:
violations_df.to_csv('parking_violations_issued_fiscal_year_2023_sample.csv')